In [113]:
import pandas as pd
import numpy as np
import random
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

In [114]:
berat_badan = 60
tinggi = 176
usia = 21
jenis_kelamin = 'pria'
faktor_aktivitas = 1.55   
koreksi_umur = 0.6  
koreksi_kerangka_tubuh = 0.2

def hitung_akg(berat_badan, tinggi, usia, jenis_kelamin, faktor_aktivitas, koreksi_umur, koreksi_kerangka_tubuh):
    tinggi_m = tinggi / 100
    imt = berat_badan / (tinggi_m ** 2)
    
    if jenis_kelamin.lower() == 'pria':
        bmr = 66 + (13.7 * berat_badan) + (5 * tinggi) - (6.8 * usia)
    elif jenis_kelamin.lower() == 'wanita':
        bmr = 665 + (9.6 * berat_badan) + (1.8 * tinggi) - (4.7 * usia)
    else:
        return "Jenis kelamin tidak valid. Masukkan 'pria' atau 'wanita'."

    faktor_aktivitas = faktor_aktivitas * bmr
    faktor_umur = koreksi_umur * bmr
    faktor_tubuh = koreksi_kerangka_tubuh * bmr
    
    total_kalori = (bmr + faktor_aktivitas) - faktor_umur + faktor_tubuh
    
    karbohidrat_kalori = 0.6 * total_kalori
    protein_kalori = 0.15 * total_kalori
    lemak_kalori = 0.25 * total_kalori

    karbohidrat_gram = karbohidrat_kalori / 4
    protein_gram = protein_kalori / 4
    lemak_gram = lemak_kalori / 9

    return {
        "imt": imt,
        "bmr": bmr,
        "total_kalori": total_kalori,
        "karbohidrat_gram": karbohidrat_gram,
        "protein_gram": protein_gram,
        "lemak_gram": lemak_gram
    }

kebutuhan_gizi = hitung_akg(berat_badan, tinggi, usia, jenis_kelamin, faktor_aktivitas, koreksi_umur, koreksi_kerangka_tubuh)

print(kebutuhan_gizi)


{'imt': 19.369834710743802, 'bmr': 1625.2, 'total_kalori': 3494.1800000000003, 'karbohidrat_gram': 524.1270000000001, 'protein_gram': 131.03175000000002, 'lemak_gram': 97.06055555555557}


In [115]:
df = pd.read_csv('raw_clean.csv')

for col in ['Kalori', 'Karbohidrat', 'Protein', 'Lemak', 'Rating']:
    df[col] = df[col].astype(str).str.replace(',', '.').astype(float)

X = df[['Kalori', 'Karbohidrat', 'Protein', 'Lemak']].values
y = df['Rating'].values

kmeans = KMeans(n_clusters=5, random_state=42) 
df['Cluster'] = kmeans.fit_predict(X)

def calculate_nutrition_distance(features, target_features):
    return np.sqrt(np.sum((features - target_features) ** 2))

c:\Users\muamm\miniconda3\envs\pds\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\muamm\miniconda3\envs\pds\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [116]:
def fitness_function(combination, target_features, valid_X, cluster_labels):
    combined_features = np.sum(valid_X[list(combination)], axis=0)
    distance = calculate_nutrition_distance(combined_features, target_features)
    diversity_score = len(set(cluster_labels[list(combination)])) 
    return diversity_score / (distance + 1e-6)

# Fungsi untuk merekomendasikan makanan menggunakan algoritma genetika
def recommend_meals_ga(target_features, valid_X, cluster_labels, population_size=100, num_generations=100):
    num_meals = len(valid_X)

    population = [random.sample(range(num_meals), 10) for _ in range(population_size)]

    for generation in range(num_generations):
        fitness_scores = [fitness_function(individual, target_features, valid_X, cluster_labels) for individual in population]

        best_individual = population[np.argmax(fitness_scores)]
        best_fitness = max(fitness_scores)

        if best_fitness > 0.99:
            break

        new_population = []
        for _ in range(population_size):
            if random.random() < 0.2:
                new_individual = list(random.sample(range(num_meals), 10))
            else:
                parent1, parent2 = random.sample(population, 2)
                crossover_point = random.randint(0, 2)
                new_individual = parent1[:crossover_point] + parent2[crossover_point:]
            new_population.append(new_individual)
        population = new_population

    best_combination = best_individual
    recommended_meals = valid_X[list(best_combination)]
    return best_combination, recommended_meals


In [117]:
target_features = np.array([kebutuhan_gizi["total_kalori"], kebutuhan_gizi["karbohidrat_gram"], kebutuhan_gizi["protein_gram"], kebutuhan_gizi["lemak_gram"]])

valid_indices = np.where(np.any(X != 0, axis=1))[0]
valid_data = df.iloc[valid_indices]
valid_X = valid_data[['Kalori', 'Karbohidrat', 'Protein', 'Lemak']].values
cluster_labels = valid_data['Cluster'].values

best_combination, recommended_meals = recommend_meals_ga(target_features, valid_X, cluster_labels)
print("Total Kalori:", kebutuhan_gizi["total_kalori"], "kalori per hari")
print("Kebutuhan Karbohidrat:", kebutuhan_gizi["karbohidrat_gram"], "gram per hari")
print("Kebutuhan Protein:", kebutuhan_gizi["protein_gram"], "gram per hari")
print("Kebutuhan Lemak:", kebutuhan_gizi["lemak_gram"], "gram per hari")
print("Rekomendasi makanan dan kandungan gizinya:")
for idx in best_combination:
    meal = valid_data.iloc[idx]
    print(f"{meal['Nama']} - Kalori: {meal['Kalori']}, Karbohidrat: {meal['Karbohidrat']}, Protein: {meal['Protein']}, Lemak: {meal['Lemak']}")


Total Kalori: 3494.1800000000003 kalori per hari
Kebutuhan Karbohidrat: 524.1270000000001 gram per hari
Kebutuhan Protein: 131.03175000000002 gram per hari
Kebutuhan Lemak: 97.06055555555557 gram per hari
Rekomendasi makanan dan kandungan gizinya:
Pisang Siam goreng - Kalori: 625.0, Karbohidrat: 8.1, Protein: 1.0, Lemak: 0.0
Kalio kikil (tunjang)  - Kalori: 122.0, Karbohidrat: 3.9, Protein: 13.8, Lemak: 5.7
Bagea kelapa manis - Kalori: 452.0, Karbohidrat: 80.6, Protein: 1.3, Lemak: 13.8
